In [ ]:
from datasets import load_from_disk

dataset = load_from_disk('/home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_train')

def prompt_replace(example):
    example['prompt'] = example['prompt'] + '\nReturn the code within ```python ... ``` markup. Write unit tests for the code with the standard input/output like this: <test_input>(stdin)</test_input><test_output>(stdout)</test_output> tags. If you make multiple unit tests, use the <test_input> ... </test_input><test_output> ... </test_output> tags to write multiple unit tests one after another.'
    return example

dataset2 = dataset.map(prompt_replace)

!rm -r '/home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_train_with_unit_tests_v2'
dataset2.save_to_disk('/home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_train_with_unit_tests_v2')


In [ ]:
# Shuffle the dataset
import random

# Set a random seed for reproducibility
random_seed = 42
random.seed(random_seed)

# Create a shuffled version of the dataset
shuffled_dataset = dataset2.shuffle(seed=random_seed)

# Calculate split sizes
dataset_size = len(shuffled_dataset)
train_size = int(dataset_size * 0.8)
val_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - val_size

# Split the dataset
train_dataset = shuffled_dataset.select(range(0, train_size))
val_dataset = shuffled_dataset.select(range(train_size, train_size + val_size))
test_dataset = shuffled_dataset.select(range(train_size + val_size, dataset_size))

# Print split information
print(f"Dataset size: {dataset_size}")
print(f"Train split: {len(train_dataset)} examples ({len(train_dataset)/dataset_size:.1%})")
print(f"Validation split: {len(val_dataset)} examples ({len(val_dataset)/dataset_size:.1%})")
print(f"Test split: {len(test_dataset)} examples ({len(test_dataset)/dataset_size:.1%})")

# Save the splits
train_dataset.save_to_disk('/home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_with_unit_tests_v2_train')
val_dataset.save_to_disk('/home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_with_unit_tests_v2_sft')
test_dataset.save_to_disk('/home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_with_unit_tests_v2_test')


In [38]:
val_dataset = load_from_disk('/home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_with_unit_tests_v2_sft')


In [ ]:
val_dataset[100]

In [ ]:
sft_unformatted = val_dataset

def format_response(example):
    test_cases = eval(example['verification_info'])['test_cases']
    random.shuffle(test_cases)
    test_case = test_cases[:3]
    if '```python' in example['gold_standard_solution']:
        example['response'] = example['gold_standard_solution'] + "\n\n" + "### Unit Tests\n\n" + "\n".join([f"<test_input>{test_case['input']}</test_input><test_output>{test_case['output']}</test_output>" for test_case in test_case])
    else:
        example['response'] = "```python\n" + example['gold_standard_solution'] + "\n```" + "\n\n" + "### Unit Tests\n\n" + "\n".join([f"<test_input>{test_case['input']}</test_input><test_output>{test_case['output']}</test_output>" for test_case in test_case])
    return example

sft_formatted = sft_unformatted.map(format_response)

print(sft_formatted[0]['response'])


In [ ]:
!rm -r /home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_with_unit_tests_v2_sft_formatted
sft_formatted.save_to_disk('/home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_with_unit_tests_v2_sft_formatted')

In [ ]:
sanity_check = load_from_disk('/home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_with_unit_tests_v2_sft_formatted')
print(sanity_check[310]['response'])

In [ ]:
few_shot_example = """
For example, code to compute the sum of two numbers should be formatted as follows:

```python
def sum():
    a, b = map(int, input().split())
    return a + b
```

### Unit Tests

<test_input>1 1</test_input><test_output>3</test_output>
<test_input>-1 1</test_input><test_output>0</test_output>

For example, code to compute the max of a list should be formatted as follows:

```python
def max_of_list():
    return max(list(map(int, input().split())))
```

### Unit Tests

<test_input>1 2 3</test_input><test_output>3</test_input>
<test_input>-1 1 2</test_input><test_output>2</test_input>
"""

def add_few_shot_example(example):
    example['prompt'] = example['prompt'] + '\n\n' + few_shot_example
    return example

train_dataset = load_from_disk('/home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_with_unit_tests_v2_train')
train_dataset = train_dataset.map(add_few_shot_example)

train_dataset.save_to_disk('/home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_with_unit_tests_v2_train_few_shot')

In [ ]:
from transformers import AutoTokenizer

sanity_check = load_from_disk('/home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_with_unit_tests_v2_train_few_shot')
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-Coder-0.5B-Instruct")
lens = [len(tokenizer.encode(example['prompt'])) for example in sanity_check]

import matplotlib.pyplot as plt

plt.hist(lens, bins=100)
plt.show()




In [ ]:
dataset = load_from_disk('/home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_with_unit_tests_v2_sft_formatted')

print(dataset[0]['response'])

In [ ]:
dataset = load_from_disk('/home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_with_unit_tests_v2_sft_formatted')



In [ ]:
print(dataset[10])





In [55]:
# add correct implementation to check unit tests

In [3]:
from datasets import load_from_disk

train_dataset = load_from_disk('/home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_with_unit_tests_v2_train')



In [ ]:
import json

def add_reference_implementation(example):
    verification_info = eval(example['verification_info'])
    verification_info['reference_implementation'] = example['gold_standard_solution']
    example['verification_info'] = json.dumps(verification_info)
    return example

train_dataset = train_dataset.map(add_reference_implementation)
train_dataset.save_to_disk('/home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_with_unit_tests_v2_train_with_reference')



In [ ]:
print(train_dataset[0]['prompt'])



In [26]:
from datasets import Dataset, load_from_disk
import re
total = 0
match = 0

new_samples = []

train_dataset = load_from_disk('/home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_with_unit_tests_v2_train_with_reference')

patterns = [
        (r'\nSample Input.*?Now solve', 'Now solve'),
        (r'\nExample.*?Note', 'Note'),
        (r'\nExample.*?Now', 'Now'),
        (r'-Example.*?Note', 'Note'),
        (r'-Example.*?The input', 'The input'),
        (r'-Example.*?Now', 'Now'),
        (r'-\w?Sample Input.*?EXPLANATION', 'EXPLANATION'),
        (r'-Sample Input.*?The input', 'The input'),
        (r'- Sample Input.*?Return', 'Return'),
        (r'\nSample input.*?The input', 'The input'),
        (r'-Sample input.*?The input', 'The input'),
        (r'-Sample.*?The input', 'The input'),
        (r'- Sample Input.*?The input', 'The input'),
        (r'Примеры.*?The input', 'The input'),
        (r'Примеры.*?Примечание', ''),
        (r'SAMPLE INPUT.*?The input', 'The input'),
        (r'Sample Output.*?The input', 'The input'),
        (r'- Example.*?The input', 'The input'),
        (r'Input example.*?Now', 'Now'),
        ("You have one chip and one chance to play roulette. Are you feeling lucky?", "You have one chip and one chance to play roulette. Are you feeling lucky?"),
        ("fourier-doodles.zip", "fourier-doodles.zip"),
    ]
for sample in train_dataset:
    total += 1 
    for _ in [1, 2, 3, 4, 5]:
        og_prompt = sample['prompt']
        for pattern, replacement in patterns:
            if re.search(pattern, sample['prompt'], re.DOTALL):
                sample['prompt'] = re.sub(pattern, replacement, sample['prompt'], flags=re.DOTALL)
                if sample['prompt'] == og_prompt:
                    if 'fourier-doodles.zip' in og_prompt or "You have one chip and one chance to play roulette. Are you feeling lucky?" in og_prompt:
                        pass
                    else:
                        print("pattern: ", pattern)
                        print("og_prompt: ", og_prompt)
                        assert False
                match += 1
                break
    
    for pattern, replacement in patterns:
        if re.search(pattern, sample['prompt'], re.DOTALL):
            if 'fourier-doodles.zip' in og_prompt or "You have one chip and one chance to play roulette. Are you feeling lucky?" in og_prompt:
                pass
            else:
                print(pattern)

                og_lines = og_prompt.split('\n')
                prompt_lines = sample['prompt'].split('\n')

                for line1, line2 in zip(og_lines, prompt_lines):
                    print(line1)
                    print(line2)
                    print('*' * 20)

                print(prompt)
                assert False
                break
    new_samples.append(sample)


no_examples_train_set = Dataset.from_list(new_samples)

!rm -r '/home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_with_unit_tests_v2_train_with_reference_no_examples'
no_examples_train_set.save_to_disk('/home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_with_unit_tests_v2_train_with_reference_no_examples')

print(match / total, match, total, len(train_dataset))


Saving the dataset (3/3 shards): 100%|██████████| 6065/6065 [00:00<00:00, 10544.19 examples/s]

1.0511129431162407 6375 6065 6065


In [25]:
from datasets import load_from_disk

no_examples_train_set = load_from_disk('/home/rosmine/data2/rl_codegen/datasets/pi_verifiable_no_fn_call_with_unit_tests_v2_train_with_reference_no_examples')

seen = 0

for sample in no_examples_train_set:
    if 'input example' in sample['prompt'].lower():
        seen += 1
    elif 'for example' in sample['prompt'].lower():
        pass
    elif 'example' in sample['prompt'].lower():
        print(sample['prompt'])
    

print(seen)


Solve the following coding problem using the programming language python:

On her way to programming school tiger Dasha faced her first test — a huge staircase!

<image>

The steps were numbered from one to infinity. As we know, tigers are very fond of all striped things, it is possible that it has something to do with their color. So on some interval of her way she calculated two values — the number of steps with even and odd numbers. 

You need to check whether there is an interval of steps from the l-th to the r-th (1 ≤ l ≤ r), for which values that Dasha has found are correct.

Input

In the only line you are given two integers a, b (0 ≤ a, b ≤ 100) — the number of even and odd steps, accordingly.

Output

In the only line print "YES", if the interval of steps described above exists, and "NO" otherwise.
Note

In the first example one of suitable intervals is from 1 to 5. The interval contains two even steps — 2 and 4, and three odd: 1, 3 and 5.

The input will be given via stdin an